# Ikea scraper

## Installation

In [1]:
#!pip install selenium

### Driver

In [2]:
# it is necessary to add the driver to the PATH variable
import os

os.environ['PATH'] = f'{os.environ["PATH"]}:{os.getcwd()}/drivers'

In [3]:
os.environ['PATH']

'/Users/victorgalan/miniconda3/envs/proj_env/bin:/Users/victorgalan/miniconda3/condabin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin:/Library/Apple/usr/bin:/Users/victorgalan/miniconda3/bin:/Users/victorgalan/miniconda3/condabin:/Users/victorgalan/ironhack/git/projects/Ironhack-Final-Project---FurniTour_31032020/Notebooks/ikea_scraper/drivers'

I have downloaded and installed geckodriver for MacOS from https://github.com/mozilla/geckodriver/releases

## Imports:

In [4]:
from selenium import webdriver
import time 
import numpy as np
import pandas as pd

from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import *
from selenium.webdriver.support.ui import WebDriverWait



import urllib.request as ulib
from bs4 import BeautifulSoup
import ast

import regex as re

### Getting some fresh Ikea pics

In [5]:
shop = 'ikea'
product = 'Sillas'
directory = 'img_' + shop + '_' + product

In [6]:
directory

'img_ikea_Sillas'

In [7]:
''' #trials with DBR:
user_selected_product = 'http://myapp.net/search?product=Sillas'
'''

" #trials with DBR:\nuser_selected_product = 'http://myapp.net/search?product=Sillas'\n"

In [8]:
'''product = user_selected_product.replace('http://myapp.net/search?product=','')
product'''

"product = user_selected_product.replace('http://myapp.net/search?product=','')\nproduct"

## Launch Firefox:

In [9]:
driver = webdriver.Firefox()

url_ikea = 'https://www.ikea.com/es/es/cat/productos-products/'
driver.get(url_ikea)

In [10]:
if product == 'Sillas':
    product_tag = 'Todas las sillas'

link = driver.find_element_by_link_text(product_tag)    
link.click()

time.sleep(1)


try:
    count = 0
    while True:
        continue_link = driver.find_element_by_partial_link_text('Cargar más')
        continue_link.click()
        time.sleep(1)
        count += 1
        print(f'Load more: {count}')
    pass
    print('Reached the last element')


except:
    pass
    
finally:
    print('Resuming')
    pass

Load more: 1
Load more: 2
Load more: 3
Load more: 4
Load more: 5
Load more: 6
Load more: 7
Load more: 8
Load more: 9
Load more: 10
Load more: 11
Load more: 12
Load more: 13
Load more: 14
Load more: 15
Load more: 16
Load more: 17
Load more: 18
Load more: 19
Load more: 20
Load more: 21
Load more: 22
Load more: 23
Load more: 24
Load more: 25
Load more: 26
Load more: 27
Load more: 28
Load more: 29
Load more: 30
Load more: 31
Load more: 32
Load more: 33
Load more: 34
Load more: 35
Load more: 36
Load more: 37
Load more: 38
Load more: 39
Resuming


In [11]:
# Take urls of all the products images:

In [12]:
page = driver.page_source
soup = BeautifulSoup(page, 'html.parser')


image_list = []

for link in soup.find_all('img'):
    src = link.get('src')
    if src.startswith('https://www.ikea.com/es/es/images/products/'): #check src starts with...
        image_list.append(src)
        
image_list[0:5]


['https://www.ikea.com/es/es/images/products/fanbyn-chair-white__0587379_PE672586_S5.JPG?f=xs',
 'https://www.ikea.com/es/es/images/products/leifarne-chair__0745133_PE743594_S5.JPG?f=xs',
 'https://www.ikea.com/es/es/images/products/ekedalen-chair-white-orrsta-light-grey__0728310_PE736178_S5.JPG?f=xs',
 'https://www.ikea.com/es/es/images/products/odger-swivel-chair-white-beige__0734069_PE739278_S5.JPG?f=xs',
 'https://www.ikea.com/es/es/images/products/torparoe-armchair-in-outdoor-white-beige__0812003_PE771902_S5.JPG?f=xs']

In [13]:
# Take the urls to buy all the products:

In [14]:
buy_url_list = []
buy_url_list_unique = []


all_a = soup.select('.product-compact__spacer a')

for a in all_a:
    buy_url_list.append(a['href'])
        
for i in buy_url_list:
    if i not in buy_url_list_unique:
        if 'openPopup=fiche' not in i:
            buy_url_list_unique.append(i)

buy_url_list_unique[0:5]

['https://www.ikea.com/es/es/p/fanbyn-silla-blanco-s49228474/',
 'https://www.ikea.com/es/es/p/leifarne-silla-amarillo-oscuro-ernfrid-abedul-s69304216/',
 'https://www.ikea.com/es/es/p/ekedalen-silla-blanco-orrsta-gris-claro-60341015/',
 'https://www.ikea.com/es/es/p/odger-silla-giratoria-blanco-beige-70308685/',
 'https://www.ikea.com/es/es/p/torparo-silla-con-reposabrz-int-ext-blanco-beige-90420761/']

In [15]:
# Take the name of all products:

In [16]:
product_names_list = []

names = soup.select('div.product-compact__spacer span.product-compact__name')
for name in names:
    product_names_list.append(name.text)
    
product_names_list[0:5]


['FANBYN', 'LEIFARNE', 'EKEDALEN', 'ODGER', 'TORPARÖ']

In [17]:
# Take the type of product:

In [18]:
product_types_list = []

types = soup.select('div.product-compact__spacer span.product-compact__type')
for type_ in types:
    product_types_list.append(((type_.text).replace('\n','')).strip().replace('         ',''))

product_types_list[0:5]


['Silla', 'Silla', 'Silla', 'Silla giratoria', 'Silla con reposabrz int/ext']

In [19]:
# Take the price of every product:

In [20]:
product_price_list = []

prices = soup.select('div.product-compact__spacer span.product-compact__price__value')
for price in prices:
    product_price_list.append(price.text)

product_price_list[0:5]

['45€', '55€', '45€', '89€', '25€']

In [21]:
# Create field image path

In [45]:
images_path_list = []

imgs_path = soup.select('div.product-compact__spacer span.product-compact__name')
for i,img_path in enumerate(imgs_path):
    path_0 = '../data/'
    path_1 = os.path.join(directory,'{:04}.jpg'.format(i))
    path = path_0 + path_1
    images_path_list.append(path)

## Create function to save scrapped images

In [23]:
def save_img(url,directory):
    if not os.path.isdir(directory):
        os.mkdir(directory)

    for i,link in enumerate(url):
        path = os.path.join(directory,'{:04}.jpg'.format(i))
        try:
            ulib.urlretrieve(link,path)
        except:
            print('Failed: ')

# save_img(image_list,directory)

# Check the lengths of the lists:

In [24]:
len(image_list)

944

In [25]:
len(buy_url_list_unique)

944

In [26]:
len(product_names_list)

944

In [27]:
len(product_types_list)

944

In [28]:
len(product_price_list)

944

In [46]:
len(images_path_list)

944

In [47]:
scraped_dict = {'Type':product_types_list,'Local_Path':images_path_list,'Name':product_names_list,'Price':product_price_list,'Image_url':image_list,'Buy_url':buy_url_list_unique}

# Create DataFrame:

In [48]:
furnitures_df = pd.DataFrame(scraped_dict)

In [49]:
furnitures_df

,Type,Local_Path,Name,Price,Image_url,Buy_url
0,Silla,../data/img_ikea_Sillas/0000.jpg,FANBYN,45€,https://www.ikea.com/es/es/images/products/fan...,https://www.ikea.com/es/es/p/fanbyn-silla-blan...
1,Silla,../data/img_ikea_Sillas/0001.jpg,LEIFARNE,55€,https://www.ikea.com/es/es/images/products/lei...,https://www.ikea.com/es/es/p/leifarne-silla-am...
2,Silla,../data/img_ikea_Sillas/0002.jpg,EKEDALEN,45€,https://www.ikea.com/es/es/images/products/eke...,https://www.ikea.com/es/es/p/ekedalen-silla-bl...
3,Silla giratoria,../data/img_ikea_Sillas/0003.jpg,ODGER,89€,https://www.ikea.com/es/es/images/products/odg...,https://www.ikea.com/es/es/p/odger-silla-girat...
4,Silla con reposabrz int/ext,../data/img_ikea_Sillas/0004.jpg,TORPARÖ,25€,https://www.ikea.com/es/es/images/products/tor...,https://www.ikea.com/es/es/p/torparo-silla-con...
...,...,...,...,...,...,...
939,Mecedora,../data/img_ikea_Sillas/0939.jpg,POÄNG,159€,https://www.ikea.com/es/es/images/products/poa...,https://www.ikea.com/es/es/p/poang-mecedora-ch...
940,"Mesa con 4 sillas, 145 cm",../data/img_ikea_Sillas/0940.jpg,MÖRBYLÅNGA / TOSSBERG,1.215€,https://www.ikea.com/es/es/images/products/moe...,https://www.ikea.com/es/es/p/morbylanga-tossbe...
941,"Mesa con 4 sillas, 130/190x80 cm",../data/img_ikea_Sillas/0941.jpg,LANEBERG / KARLJAN,285€,https://www.ikea.com/es/es/images/products/lan...,https://www.ikea.com/es/es/p/laneberg-karljan-...
942,"Mesa con 4 sillas, 152/223x95 cm",../data/img_ikea_Sillas/0942.jpg,NORDVIKEN / RÖNNINGE,615€,https://www.ikea.com/es/es/images/products/nor...,https://www.ikea.com/es/es/p/nordviken-ronning...


In [50]:
# display only 'Sillas'
furnitures_df[furnitures_df['Type']=='Silla']

,Type,Local_Path,Name,Price,Image_url,Buy_url
0,Silla,../data/img_ikea_Sillas/0000.jpg,FANBYN,45€,https://www.ikea.com/es/es/images/products/fan...,https://www.ikea.com/es/es/p/fanbyn-silla-blan...
1,Silla,../data/img_ikea_Sillas/0001.jpg,LEIFARNE,55€,https://www.ikea.com/es/es/images/products/lei...,https://www.ikea.com/es/es/p/leifarne-silla-am...
2,Silla,../data/img_ikea_Sillas/0002.jpg,EKEDALEN,45€,https://www.ikea.com/es/es/images/products/eke...,https://www.ikea.com/es/es/p/ekedalen-silla-bl...
5,Silla,../data/img_ikea_Sillas/0005.jpg,NORDMYRA,"24,99€",https://www.ikea.com/es/es/images/products/nor...,https://www.ikea.com/es/es/p/nordmyra-silla-bl...
6,Silla,../data/img_ikea_Sillas/0006.jpg,TEODORES,20€,https://www.ikea.com/es/es/images/products/teo...,https://www.ikea.com/es/es/p/teodores-silla-bl...
...,...,...,...,...,...,...
785,Silla,../data/img_ikea_Sillas/0785.jpg,EKEDALEN,55€,https://www.ikea.com/es/es/images/products/eke...,https://www.ikea.com/es/es/p/ekedalen-silla-ne...
786,Silla,../data/img_ikea_Sillas/0786.jpg,EKEDALEN,55€,https://www.ikea.com/es/es/images/products/eke...,https://www.ikea.com/es/es/p/ekedalen-silla-ma...
841,Silla,../data/img_ikea_Sillas/0841.jpg,HENRIKSDAL,59€,https://www.ikea.com/es/es/images/products/hen...,https://www.ikea.com/es/es/p/henriksdal-silla-...
870,Silla,../data/img_ikea_Sillas/0870.jpg,HENRIKSDAL,59€,https://www.ikea.com/es/es/images/products/hen...,https://www.ikea.com/es/es/p/henriksdal-silla-...


In [51]:
# Filtered final
furnitures_df_filtered = furnitures_df[furnitures_df['Type'].str.startswith("Silla")]
furnitures_df_filtered

,Type,Local_Path,Name,Price,Image_url,Buy_url
0,Silla,../data/img_ikea_Sillas/0000.jpg,FANBYN,45€,https://www.ikea.com/es/es/images/products/fan...,https://www.ikea.com/es/es/p/fanbyn-silla-blan...
1,Silla,../data/img_ikea_Sillas/0001.jpg,LEIFARNE,55€,https://www.ikea.com/es/es/images/products/lei...,https://www.ikea.com/es/es/p/leifarne-silla-am...
2,Silla,../data/img_ikea_Sillas/0002.jpg,EKEDALEN,45€,https://www.ikea.com/es/es/images/products/eke...,https://www.ikea.com/es/es/p/ekedalen-silla-bl...
3,Silla giratoria,../data/img_ikea_Sillas/0003.jpg,ODGER,89€,https://www.ikea.com/es/es/images/products/odg...,https://www.ikea.com/es/es/p/odger-silla-girat...
4,Silla con reposabrz int/ext,../data/img_ikea_Sillas/0004.jpg,TORPARÖ,25€,https://www.ikea.com/es/es/images/products/tor...,https://www.ikea.com/es/es/p/torparo-silla-con...
...,...,...,...,...,...,...
870,Silla,../data/img_ikea_Sillas/0870.jpg,HENRIKSDAL,59€,https://www.ikea.com/es/es/images/products/hen...,https://www.ikea.com/es/es/p/henriksdal-silla-...
892,Silla,../data/img_ikea_Sillas/0892.jpg,EKEDALEN,55€,https://www.ikea.com/es/es/images/products/eke...,https://www.ikea.com/es/es/p/ekedalen-silla-ro...
893,Silla+funda larga,../data/img_ikea_Sillas/0893.jpg,HENRIKSDAL,64€,https://www.ikea.com/es/es/images/products/hen...,https://www.ikea.com/es/es/p/henriksdal-silla-...
899,Silla de trabajo,../data/img_ikea_Sillas/0899.jpg,LÅNGFJÄLL,149€,https://www.ikea.com/es/es/images/products/lan...,https://www.ikea.com/es/es/p/langfjall-silla-t...


In [52]:
len(furnitures_df_filtered['Image_url'])

283

# Saves images filtered:

In [35]:
save_img(furnitures_df_filtered['Image_url'],directory)

# DataFrame info to .json file:

In [36]:
directory_json = 'json'

In [37]:
json_path = os.path.join(directory_json,f'{directory}.json')
json_path

'json/img_ikea_Sillas.json'

In [38]:
furnitures_df.to_json(json_path)

## Close Firefox

In [39]:
driver.close()